## Install missing packages

In [ ]:
pip install matplotlib

In [7]:
pip install scipy

     |████████████████████████████████| 55.6 MB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install geopandas

  Using cached geopandas-0.10.2-py2.py3-none-any.whl (1.0 MB)
     |████████████████████████████████| 18.5 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.7 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 3.3 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install plotly

     |████████████████████████████████| 27.7 MB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Import relevant libraries

In [1]:
import pandas as pd
import scipy as sp
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
import geoplotlib

def inline(width=900):
    """display the map inline in ipython
    :param width: image width for the browser
    """
    from IPython.display import Image, HTML, display, clear_output
    import random
    import string
    import urllib
    import os
    import base64

    while True:
        fname = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(29))
        if not os.path.isfile(fname + '.png'):
            break

    geoplotlib.savefig(fname)

    if os.path.isfile(fname + '.png'):
        with open(fname + '.png', 'rb') as fin:
            encoded = base64.b64encode(fin.read())
            b64 = urllib.parse.quote(encoded)

        image_html = "<img style='width: %dpx; margin: 0px; float: left; border: 1px solid black;' src='data:image/png;base64,%s' />" % (width, b64)

        display(HTML(image_html))
        os.remove(fname + '.png')

geoplotlib.inline = inline

In [3]:
#Importing geoplotlib
import geoplotlib
from geoplotlib.utils import BoundingBox
from geoplotlib.utils import DataAccessObject
import json
import pandas as pd
import numpy as np
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.utils import BoundingBox
from geoplotlib.colors import ColorMap

# tower.png's

In [4]:
#First, load the csv file, and filter for the columns 
#['LATITUDE','LONGITUDE','TRANSMIT_FREQ', 'TX_PWR', 'SITE_ELEV', 'TX_ANT_HT', 'TX_ANT_AZIM', 'TX_ANT_ELEV_ANGLE', 'TX_ANT_GAIN']
datasetTowers = pd.read_csv("Site_Data_Extract.csv", encoding='latin-1',low_memory=False)
datasetTowers = datasetTowers[['LATITUDE',
                               'LONGITUDE',
                               'TRANSMIT_FREQ',
                               'TX_PWR',
                               'SITE_ELEV',
                               'TX_ANT_HT',
                               'TX_ANT_AZIM',
                               'TX_ANT_ELEV_ANGLE',
                               'TX_ANT_GAIN']]

datasetTowers.dropna(inplace=True) #Then remove nan values from the dataframe using the function df.dropna()
datasetTowers.rename({'LATITUDE':'lat', 'LONGITUDE':'lon'}, axis=1, inplace=True) #Rename latitute and longitude

In [5]:
# custom layer creation
import geoplotlib
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox

class MyLayer(BaseLayer):

    def __init__(self, dataset, feature, bbox=BoundingBox.USA):
        self.data = dataset
        self.feature = feature
        self.cmap = ColorMap('jet', alpha=250, levels= 20)
        self.painter = BatchPainter()
        self.view = bbox
   
    def invalidate(self, proj):
        self.painter = BatchPainter()

        for idx, element in enumerate(self.data[self.feature]):
            scale = 'lin' if self.feature == 'TX_ANT_ELEV_ANGLE' else 'sqrt'
            elementColor = self.cmap.to_color(np.round(element), maxvalue= self.data[self.feature].max(), scale=scale)
            self.painter.set_color(elementColor)
    
            lat, lon = self.data['lon'][idx], self.data['lat'][idx] # get the lon and lat points (self.data['lon'][index], self.data['lat'][index])
            x, y = proj.lonlat_to_screen(lat, lon) # call the function proj.lonlat_to_screen
            self.painter.points(x, y, 5, rounded=False) # call the function painter.points

    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter.batch_draw()
        ui_manager.info(f'Current plot: {self.feature}')

    # bounding box that gets used when layer is created
    def bbox(self):
        return self.view

In [6]:
from geoplotlib.utils import DataAccessObject
dataAccessTowers = DataAccessObject(datasetTowers)

from geoplotlib.utils import BoundingBox
bbCanada = BoundingBox.from_nominatim('CANADA')

for var in ['TRANSMIT_FREQ', 'TX_PWR', 'SITE_ELEV', 'TX_ANT_HT', 'TX_ANT_AZIM', 'TX_ANT_ELEV_ANGLE', 'TX_ANT_GAIN']:
    geoplotlib.tiles_provider('darkmatter')
    geoplotlib.add_layer(MyLayer(dataAccessTowers, feature=var, bbox=bbCanada))
    geoplotlib.inline(1000)

('bbox from Nominatim:', 41.6765556, 83.3362128, -141.00275, -52.3231981)


2022-03-09 23:35:48.910 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cb04860>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:35:48.911 python[37451:1362258] Warning: Expected min height of view: (<NSButton: 0x7ff01cb68a40>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:35:48.913 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cb6bd30>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:35:48.915 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cb74570>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


Traceback (most recent call last):
  File "/Users/mmccoy/opt/anaconda3/envs/new_DATA/lib/python3.9/site-packages/geoplotlib/__init__.py", line 32, in _runapp
    app.start()
  File "/Users/mmccoy/opt/anaconda3/envs/new_DATA/lib/python3.9/site-packages/geoplotlib/core.py", line 367, in start
    l.invalidate(self.proj)
  File "/var/folders/qy/bxzjv74x1_35ytn68g0wnd8w0000gn/T/ipykernel_37451/353706152.py", line 22, in invalidate
    elementColor = self.cmap.to_color(np.round(element), maxvalue= self.data[self.feature].max(), scale=scale)
  File "/Users/mmccoy/opt/anaconda3/envs/new_DATA/lib/python3.9/site-packages/geoplotlib/colors.py", line 47, in to_color
    raise Exception('value and maxvalue must be greater than 0')
Exception: value and maxvalue must be greater than 0
2022-03-09 23:38:32.561 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cc2d5d0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logge

2022-03-09 23:41:02.548 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cbf7f90>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:41:02.548 python[37451:1362258] Warning: Expected min height of view: (<NSButton: 0x7ff01cc7bff0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:41:02.550 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cc81cf0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:41:02.552 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cc88720>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


2022-03-09 23:43:51.003 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cc24e20>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:43:51.003 python[37451:1362258] Warning: Expected min height of view: (<NSButton: 0x7ff01ccbfc60>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:43:51.005 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01ccc28a0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:43:51.007 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01ccc9550>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


2022-03-09 23:46:26.416 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01ccca5b0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:46:26.416 python[37451:1362258] Warning: Expected min height of view: (<NSButton: 0x7ff01ccb6740>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:46:26.419 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01ccbbd90>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-03-09 23:46:26.420 python[37451:1362258] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7ff01cccbab0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
